# Train Qwen2.5-0.5B Chess Model on 108K Dataset

This notebook trains a 0.5B parameter model on 108K chess positions with:
- Diverse positions (opening, middlegame, endgame)
- 8K Lichess puzzles
- Material, mobility, and top-5 move evaluations
- Stockfish depth 7 analysis

In [ ]:
import os, platform
print("CWD:", os.getcwd())
print("Python:", platform.python_version())

try:
    import torch
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))
        print("GPU Memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")
except ImportError:
    print("torch not installed yet")

In [ ]:
# Install dependencies
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers accelerate datasets peft trl python-chess bitsandbytes hf-transfer
!pip install flash-attn --no-build-isolation

In [ ]:
import os

print("Initial contents in current directory:")
for name in os.listdir():
    print("-", name)

# Clone repo if not present
if not os.path.exists('train_scripts/train.py'):
    if not os.path.exists('sft'):
        print('Cloning repo Bot-Rakshit/sft...')
        !git clone https://github.com/Bot-Rakshit/sft.git
    os.chdir('sft')
    print('Changed directory to:', os.getcwd())

print('\nRepo contents:')
for name in os.listdir():
    print("-", name)

assert os.path.exists('train_scripts/train.py'), 'train_scripts/train.py not found'

In [ ]:
# Verify training data
!wc -l train_data_108k_complete.jsonl
!head -n 1 train_data_108k_complete.jsonl

In [ ]:
# Optional: Create smaller subset for faster testing (10K examples, ~5-10 min)
# Uncomment to use smaller dataset first
# !head -n 10000 train_data_108k_complete.jsonl > train_data_10k.jsonl
# !wc -l train_data_10k.jsonl

In [ ]:
# Train Qwen2.5-0.5B with LoRA on full 108K dataset
# Optimized for A100 GPU:
#   - Batch size 16 with grad_accum 2 = effective batch 32
#   - Flash Attention 2 for 2-3x speedup
#   - BF16 precision for faster computation
# Expected time: ~20-30 minutes on A100
!python train_scripts/train.py \
  --model Qwen/Qwen2.5-0.5B-Instruct \
  --data train_data_108k_complete.jsonl \
  --output qwen-chess-0.5b-108k \
  --epochs 1 \
  --batch-size 16 \
  --grad-accum 2 \
  --lr 1.5e-4 \
  --max-seq-length 512 \
  --dtype bf16 \
  --use-flash-attn

In [ ]:
# Merge LoRA adapter with base model
!python train_scripts/merge_model.py \
  --base-model Qwen/Qwen2.5-0.5B-Instruct \
  --adapter qwen-chess-0.5b-108k \
  --output qwen-chess-0.5b-108k-merged

In [ ]:
# Test the merged model
!python train_scripts/test_model.py \
  --model qwen-chess-0.5b-108k-merged \
  --base-model Qwen/Qwen2.5-0.5B-Instruct

In [ ]:
# Save to Google Drive (optional)
# Uncomment to mount and save
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r qwen-chess-0.5b-108k-merged /content/drive/MyDrive/

# Or create a zip for download
!zip -r qwen-chess-0.5b-108k-merged.zip qwen-chess-0.5b-108k-merged
print("\nModel saved as qwen-chess-0.5b-108k-merged.zip")
print("Download from the Files panel on the left")

## Training Complete!

Your model has been trained on 108K positions and is ready for evaluation.